In [146]:
import nltk
from nltk.tokenize import RegexpTokenizer

In [147]:
query = "SELECT * FROM orders WHERE order_date >= '$start_date' AND order_total < $max_total AND order_status = $status AND order_id IN ($order_ids) AND order_amount BETWEEN :min_amount AND $max_amount AND customer_id = $customer_id AND order_is_active = $is_active"

In [148]:
pattern = r'\$?\w+'
tokenizer = RegexpTokenizer(pattern)
tokens = tokenizer.tokenize(query)

In [149]:
def sanitize(token):
   return token.replace("'", "''")

In [150]:
variables = []
for token in tokens:
    if token.startswith("$") and query.index(token) > query.index("WHERE"):
        datatype = "int"
        if "date" in token:
            datatype = "date"
        if token.startswith("$is"):
            datatype = "bool"
        if query[query.index(token) - 1] in ["'", "\""]:
            datatype = "str"
        variables.append((token, datatype))


In [151]:
pstmt = query
for name, datatype in variables:
    placeholder = "?"
    if datatype == "date":
        placeholder = "TO_DATE(?, 'YYYY-MM-DD')"
    pstmt = pstmt.replace(name, placeholder, 1)

In [152]:
print("Statement type", tokens[0])
print(pstmt)
for i, var in enumerate(variables):
    print("bind(" + var[0] + ", " + str(i) + ")")

Statement type SELECT
SELECT * FROM orders WHERE order_date >= '?' AND order_total < ? AND order_status = ? AND order_id IN (?) AND order_amount BETWEEN :min_amount AND ? AND customer_id = ? AND order_is_active = ?
bind($start_date, 0)
bind($max_total, 1)
bind($status, 2)
bind($order_ids, 3)
bind($max_amount, 4)
bind($customer_id, 5)
bind($is_active, 6)
